In [1]:
import csv
import copy
from tqdm import tqdm
import pandas as pd
import numpy as np

In [146]:
csv_file = "data/stock_clean.csv"
df = pd.read_csv(csv_file)

In [147]:
df = df.rename(columns = {df.columns[0]:"date"})

In [148]:
df["date"]

0      19620831
1      19620928
2      19621031
3      19621130
4      19621231
         ...   
391    19950331
392    19950428
393    19950531
394    19950630
395    19950731
Name: date, Length: 396, dtype: int64

In [149]:
result = df["date"].isin(["19720731"])
t = df["date"][result].index.values[0]+1
print(t)

120


In [173]:
tmp_df = df[t-119:t+1].dropna(thresh=df[t-119:t+1].shape[0],how='all',axis=1)
#print(tmp_df)
N = len(tmp_df.columns[1:])
q = 0.01
print(tmp_df[tmp_df.columns[351]][29])

C


In [164]:
cov_matrix = tmp_df[tmp_df.columns[1:]].values
print(np.where(cov_matrix=="C"))

(array([28, 33, 44, 66, 78]), array([350, 437, 437,  94, 499]))


In [138]:
cov_matrix_inv = pd.DataFrame(np.linalg.pinv(cov_matrix.values), cov_matrix.columns, cov_matrix.index)
print(cov_matrix_inv)

             AB         FTR          DR         BRY         AIN           G  \
AB   186.391981  -46.748916    3.547736  -48.263696  -22.673003   -3.968849   
FTR  -46.748916  289.805443  -38.851440    3.163909  -36.193252   -6.878656   
DR     3.547736  -38.851440  480.949793  -18.733520   28.546617   -5.379183   
BRY  -48.263696    3.163909  -18.733520  432.759294  -20.102290 -118.408700   
AIN  -22.673003  -36.193252   28.546617  -20.102290  128.307103   24.301630   
G     -3.968849   -6.878656   -5.379183 -118.408700   24.301630  377.967941   
PET  -41.642000  -26.998050   18.506655  -28.390981   -0.292684    6.982861   
FOX  -38.159383    4.983516  -42.858913  -15.593645  -23.618009  -28.064448   
WHX   -1.980789  -43.907969  -13.455402   -9.242958    5.934511  -10.439643   
BLI  -17.588704    2.859181  -72.215497  -15.243481   -0.518217  -46.261262   
HI   -55.347546  -35.618904  -86.459636   24.327864  -16.942663  -98.469287   
APY   -6.201241   -2.549720    0.062726    6.828299 

In [137]:
mu = tmp_df[tmp_df.columns[1:]].mean()
print(mu)

AB     0.015481
FTR    0.011091
DR     0.009605
BRY    0.013330
AIN    0.005754
G      0.008893
PET    0.011356
FOX    0.010433
WHX    0.002974
BLI    0.011685
HI     0.012216
APY    0.004591
dtype: float64


In [134]:
A = np.ones((1,N))@cov_matrix_inv.values@np.ones((N,1))
print(A)

[[663.3903902]]


In [135]:
B = np.ones((1,N))@cov_matrix_inv.values@mu.values
print(B)

[6.76293001]


In [136]:
C = mu.values.T@cov_matrix_inv.values@mu.values
print(C)

0.098079099754565


In [133]:
w = (C-q*B)/(A*C-B*B)*cov_matrix_inv.values@np.ones((N,1))+(q*A-C)/(A*C-B*B)*cov_matrix_inv.values@mu.values[:,np.newaxis]
print(np.sum(w))

3.3321121222013454


In [ ]:
for i in range(t,396):
    tmp_df = df[i-119:i+1].dropna(thresh=df[i-119:i+1].shape[0],how='all',axis=1)
    print(len(tmp_df.columns))